In [1]:
from exp.utils import *
from exp.models import *
from exp.losses import *
from tqdm.notebook import tqdm
from multiprocessing import Pool

import torch
import torch.nn as NN
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [17]:
seed = 92
seed_everything(seed)

In [44]:
def get_td_path():
    return Path(f"{os.environ.get('HOME')}/.datasets/fashion_product")

def get_td_image_path():
    return get_td_path()/"images"

def get_td_csv_path():
    return get_td_path()/"styles.csv"
    #return pd.read_csv(get_td_path()/"styles.csv", encoding="unicode")

def get_td_label_df():
    import csv
    from PIL import Image
    all_data = []
    # open annotation file
    with open(get_td_csv_path()) as csv_file:
        # parse it as CSV
        reader = csv.DictReader(csv_file)
        # tqdm shows pretty progress bar
        # each row in the CSV file corresponds to the image
        for row in tqdm(reader, total=reader.line_num):
            # we need image ID to build the path to the image file
            img_id = row['id']
            img_name = str(img_id) + ".jpg"
            # we're going to use only 3 attributes
            gender = row['gender']
            articleType = row['articleType']
            baseColour = row['baseColour']
            img_name = get_td_image_path()/img_name #os.path.join(input_folder, 'images', str(img_id) + '.jpg')
            # check if file is in place
            if os.path.exists(img_name):
                # check if the image has 80*60 pixels with 3 channels
                img = Image.open(img_name)
                if img.size == (60, 80) and img.mode == "RGB":
                    all_data.append([img_name, gender, articleType, baseColour])

    all_data = pd.DataFrame(np.asarray(all_data), columns=["Image Index", "L1", "L2", "L3"])
    labs = []
    for r_idx, row in all_data.iterrows():
        lab = [row[l] for l in ["L1", "L2", "L3"]]
        labs.append("|".join(lab))

    all_data["Labels"] = labs
    all_data = all_data.drop(columns=["L1", "L2", "L3"])
    return all_data

In [45]:
df = get_td_label_df()

In [46]:
all_data.shape

(43987, 2)

In [47]:
all_data.head()

,Image Index,Labels
0,/home/favi/.datasets/fashion_product/images/15...,Men|Shirts|Navy Blue
1,/home/favi/.datasets/fashion_product/images/39...,Men|Jeans|Blue
2,/home/favi/.datasets/fashion_product/images/21...,Men|Track Pants|Black
3,/home/favi/.datasets/fashion_product/images/53...,Men|Tshirts|Grey
4,/home/favi/.datasets/fashion_product/images/18...,Men|Tshirts|Grey


In [32]:
all_labels = []
for l in ["L1", "L2", "L3"]:
    all_labels = [*all_labels, *all_data[l].unique()]
all_labels = list(set(all_labels))
len(all_labels)

194